In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import concurrent.futures
import gc
import time

print(f"TensorFlow version: {tf.__version__}")

physical_devices = tf.config.list_physical_devices('GPU')
try:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(f"Found {len(physical_devices)} GPU(s), memory growth enabled")
except Exception as e:
    print(f"Memory growth setting error: {e}")

# Check if GPU is available and print device information
if tf.config.list_physical_devices('GPU'):
    print("GPU is available:")
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"  {gpu}")
    print(f"TensorFlow built with CUDA: {tf.test.is_built_with_cuda()}")
else:
    print("No GPU found! Running on CPU.")

# Define constants
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 64  # Increased batch size for better GPU utilization
EPOCHS = 20
CLASSES = ['basal cell carcinoma', 'dermatofibroma', 'melanoma', 'squamous cell carcinoma']
CLASS_MAPPING = {cls: i for i, cls in enumerate(CLASSES)}
AUGMENTATION_FACTOR = 20  # Create 20 versions of each dark skin image

# Function to create DataFrame from image directory
def create_dataframe(data_dir):
    data = []
    for class_name in os.listdir(data_dir):
        if class_name not in CLASSES:
            continue

        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            if os.path.isfile(img_path) and img_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                data.append({
                    "image_path": img_path,
                    "label": class_name,
                    "class_id": CLASS_MAPPING[class_name],
                    "is_augmented": False,
                    "source": os.path.basename(data_dir)
                })

    df = pd.DataFrame(data)
    print(f"Created DataFrame with {len(df)} images from {data_dir}")
    return df

# Function to remove hair from skin lesion images
def remove_hair(image):
    # Convert to grayscale
    if len(image.shape) == 3 and image.shape[2] == 3:
        grayscale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        grayscale = image

    # Create kernel for morphology operation
    kernel = cv2.getStructuringElement(1, (17, 17))

    # Apply blackhat filter
    blackhat = cv2.morphologyEx(grayscale, cv2.MORPH_BLACKHAT, kernel)

    # Apply threshold
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)

    # Inpaint
    if len(image.shape) == 3 and image.shape[2] == 3:
        result = cv2.inpaint(image, mask, 1, cv2.INPAINT_TELEA)
    else:
        result = cv2.inpaint(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), mask, 1, cv2.INPAINT_TELEA)
        result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    return result

# Efficient image loading with tf.data
def preprocess_image(image_path, label):
    """Load and preprocess an image using TensorFlow operations"""
    # Read the image file
    img = tf.io.read_file(image_path)

    # Decode the image
    img = tf.image.decode_image(img, channels=3, expand_animations=False)

    # Resize the image
    img = tf.image.resize(img, IMAGE_SIZE)

    # Normalize to [0,1] and EXPLICITLY set dtype to float32 for consistency
    img = tf.cast(img, tf.float32) / 255.0

    # Also make sure the label has consistent dtype
    label = tf.cast(label, tf.float32)

    return img, label

# Create tf.data.Dataset for efficient loading
def create_dataset(image_paths, labels, batch_size=BATCH_SIZE, is_training=True):
    """Create an optimized tf.data pipeline for image loading"""
    # Convert labels to float32 for consistency
    labels = labels.astype(np.float32)

    # Create dataset from tensors
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Map preprocessing function
    dataset = dataset.map(
        preprocess_image,
        num_parallel_calls=tf.data.AUTOTUNE  # Use multiple CPU cores
    )

    if is_training:
        # Shuffle and repeat for training
        dataset = dataset.shuffle(buffer_size=1000)

        # Add data augmentation for training
        dataset = dataset.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=tf.data.AUTOTUNE
        )

    # Batch the data
    dataset = dataset.batch(batch_size)

    # Prefetch for better performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

# TensorFlow-based data augmentation (runs on GPU)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomContrast(0.2),
])

# Process dark skin images (in batches to save memory)
def process_images_in_batches(image_paths, batch_size=50):
    """Process images in batches to avoid memory issues"""
    processed_images = []
    total_batches = len(image_paths) // batch_size + (1 if len(image_paths) % batch_size > 0 else 0)

    for i in range(total_batches):
        batch_paths = image_paths[i * batch_size:(i + 1) * batch_size]
        batch_images = []

        for path in batch_paths:
            try:
                # Load image
                img = np.array(Image.open(path).convert('RGB'))

                # Remove hair
                img_no_hair = remove_hair(img)

                # Resize
                resized_img = cv2.resize(img_no_hair, IMAGE_SIZE)

                batch_images.append(resized_img)
            except Exception as e:
                print(f"Error processing {path}: {e}")
                # Return a blank image in case of error
                batch_images.append(np.zeros((IMAGE_SIZE[0], IMAGE_SIZE[1], 3), dtype=np.uint8))

        processed_images.extend(batch_images)

        # Force garbage collection after each batch
        gc.collect()

    return processed_images

# Enhanced function to create heavily augmented versions of dark skin images
def create_augmented_dark_skin_dataset(dark_skin_df, multiplier=AUGMENTATION_FACTOR):
    """
    Create heavily augmented versions of dark skin images to increase effective dataset size

    Parameters:
    dark_skin_df: DataFrame with dark skin images
    multiplier: How many augmented versions to create per original image

    Returns:
    DataFrame with original and augmented images
    """
    print(f"Creating {multiplier}x augmented versions of {len(dark_skin_df)} dark skin images...")

    # Create a stronger augmentation pipeline specifically for dark skin
    dark_skin_datagen = ImageDataGenerator(
        # Geometric transformations
        rotation_range=45,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=[0.8, 1.2],
        horizontal_flip=True,

        # Color/intensity transformations (critical for skin tone)
        brightness_range=[0.6, 1.4],
        channel_shift_range=0.4,
        fill_mode='reflect',

        # Shear for shape variation
        shear_range=0.2
    )

    # Create new dataframe to store augmented images
    augmented_data = []

    # Process each class separately to ensure balanced augmentation
    for class_name in dark_skin_df['label'].unique():
        class_df = dark_skin_df[dark_skin_df['label'] == class_name]

        print(f"Augmenting {len(class_df)} images from class: {class_name}")

        # Process in smaller batches to avoid memory issues
        for i in range(0, len(class_df), 3):  # Even smaller batch size (3)
            batch_df = class_df.iloc[i:i+3]

            # Create augmentations for this batch
            for _, row in batch_df.iterrows():
                # Add original image
                augmented_data.append(row.to_dict())

                # Create image batch for augmentation
                img = row['image'].reshape(1, *row['image'].shape)

                # Generate augmented versions in smaller chunks to save memory
                for j in range(0, multiplier-1, 5):
                    # Create 5 augmented versions at a time (or fewer for the last chunk)
                    chunk_size = min(5, multiplier-1-j)
                    aug_generator = dark_skin_datagen.flow(img, batch_size=1)

                    for k in range(chunk_size):
                        aug_img = next(aug_generator)[0].astype('uint8')

                        # Create new row with augmented image
                        new_row = row.to_dict()
                        new_row['image'] = aug_img
                        new_row['is_augmented'] = True
                        new_row['aug_id'] = j + k + 1

                        # Add to dataset
                        augmented_data.append(new_row)

                    # Clear memory after each chunk
                    gc.collect()

            # Clear memory after each batch
            gc.collect()

    # Convert to dataframe
    result_df = pd.DataFrame(augmented_data)

    # Print augmentation summary
    print("\nAugmentation Summary:")
    class_counts = result_df.groupby('label').size()
    original_counts = dark_skin_df.groupby('label').size()

    print("-" * 50)
    print(f"{'Class':<30} {'Original':<10} {'Total':<10}")
    print("-" * 50)

    for cls in CLASSES:
        if cls in original_counts:
            orig = original_counts[cls]
            total = class_counts.get(cls, 0)
            print(f"{cls:<30} {orig:<10} {total:<10}")

    print("-" * 50)
    print(f"{'TOTAL':<30} {len(dark_skin_df):<10} {len(result_df):<10}")

    return result_df

# Function to create a DenseNet model with mixed precision
def create_model(input_shape=(128, 128, 3), num_classes=4):
    # Enable mixed precision just for the model
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    # Place the model on GPU explicitly
    with tf.device('/GPU:0'):
        # Create base DenseNet model
        base_model = DenseNet201(
            include_top=False,
            weights='imagenet',
            input_shape=input_shape,
            pooling='avg'
        )

        # Freeze early layers (transfer learning)
        for layer in base_model.layers[:-30]:
            layer.trainable = False

        # Add classification layers
        x = base_model.output
        x = Dropout(0.5)(x)
        predictions = Dense(num_classes, activation='softmax', dtype='float32')(x)

        # Create model
        model = Model(inputs=base_model.input, outputs=predictions)

        # Use SGD optimizer
        opt = SGD(learning_rate=0.001, momentum=0.9)

        # Compile model
        model.compile(
            optimizer=opt,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    return model

# Function to visualize training progress
def plot_training_history(history, title="Model Training", output_dir="."):
    """Plot the training history to monitor for GPU performance issues"""
    plt.figure(figsize=(15, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    save_path = os.path.join(output_dir, f"{title.replace(' ', '_').lower()}.png")
    plt.savefig(save_path)
    plt.show()
    print(f"Training history plot saved to {save_path}")

# Function to create and save confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names, title="Confusion Matrix", output_dir="."):
    """
    Create and save a confusion matrix visualization

    Parameters:
    y_true: True labels (class indices)
    y_pred: Predicted labels (class indices)
    class_names: List of class names for axis labels
    title: Title for the plot
    output_dir: Directory to save the output
    """
    # Create figure
    plt.figure(figsize=(10, 8))

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Create a nice visualization
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap=plt.cm.Blues, values_format='d')

    # Add titles and formatting
    plt.title(title, fontsize=16)
    plt.grid(False)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Save the figure
    save_path = os.path.join(output_dir, f"{title.replace(' ', '_').lower()}.png")
    plt.savefig(save_path)
    plt.show()
    print(f"Confusion matrix saved to {save_path}")

    return cm

# Function to create and save classification report
def save_classification_report(y_true, y_pred, class_names, title="Classification Report", output_dir="."):
    """
    Generate and save a classification report

    Parameters:
    y_true: True labels (class indices)
    y_pred: Predicted labels (class indices)
    class_names: List of class names
    title: Title for the report
    output_dir: Directory to save the output
    """
    # Generate the classification report
    report = classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        output_dict=True
    )

    # Convert to DataFrame for better formatting
    report_df = pd.DataFrame(report).transpose()

    # Print the report
    print(f"\n{title}:")
    print("-" * 70)
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Save the report to CSV
    save_path = os.path.join(output_dir, f"{title.replace(' ', '_').lower()}.csv")
    report_df.to_csv(save_path)
    print(f"Classification report saved to {save_path}")

    return report_df

# Function to plot model comparison
def plot_model_comparison(baseline_results, combined_results, class_names, output_dir="."):
    """Plot a comparison of model performances"""
    plt.figure(figsize=(14, 8))

    # Plot overall accuracy comparison
    plt.subplot(1, 2, 1)
    models = ['Baseline (ISIC only)', 'Combined (ISIC + Dark Skin)']
    accuracies = [
        baseline_results['overall']['accuracy'],
        combined_results['overall']['accuracy']
    ]
    colors = ['#3498db', '#2ecc71']

    plt.bar(models, accuracies, color=colors)
    plt.title('Overall Accuracy on Dark Skin Test Images')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1.0)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Annotate exact values
    for i, acc in enumerate(accuracies):
        plt.text(i, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

    # Plot per-class accuracy comparison
    plt.subplot(1, 2, 2)

    # Set up data
    x = np.arange(len(class_names))
    width = 0.35

    baseline_class_acc = [baseline_results['per_class'][cls]['accuracy'] for cls in class_names]
    combined_class_acc = [combined_results['per_class'][cls]['accuracy'] for cls in class_names]

    # Plot bars
    plt.bar(x - width/2, baseline_class_acc, width, label='Baseline Model', color='#3498db')
    plt.bar(x + width/2, combined_class_acc, width, label='Combined Model', color='#2ecc71')

    # Add labels and formatting
    plt.title('Per-Class Accuracy on Dark Skin Test Images')
    plt.xlabel('Class')
    plt.ylabel('Accuracy')
    plt.xticks(x, [c.split()[-1] for c in class_names], rotation=45, ha='right')
    plt.ylim(0, 1.0)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    save_path = os.path.join(output_dir, "model_comparison.png")
    plt.savefig(save_path)
    plt.show()
    print(f"Model comparison plot saved to {save_path}")

# Monitor memory usage
def process_memory_usage():
    """Return memory usage in GB"""
    import psutil
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    return memory_info.rss / (1024 ** 3)  # Convert bytes to GB

# Main function with memory-optimized approach
def main():
    start_time = time.time()

    # Define paths
    isic_train_dir = 'C:\\Users\\Priyanka Joshi\\Downloads\\ai_tech_project\\ISIC\\Train'
    isic_test_dir = 'C:\\Users\\Priyanka Joshi\\Downloads\\ai_tech_project\\ISIC\\Test'
    dark_skin_dir = 'C:\\Users\\Priyanka Joshi\\Downloads\\ai_tech_project\\darkskin'
    output_dir = 'C:\\Users\\Priyanka Joshi\Downloads\\ai_tech_project\\output'

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Create dataframes with paths only (no images yet)
    print("Creating dataframes from directories...")
    isic_train_df = create_dataframe(isic_train_dir)
    isic_test_df = create_dataframe(isic_test_dir)
    dark_skin_df = create_dataframe(dark_skin_dir)

    # Step 2: Process dark skin images (smaller dataset)
    print("Processing dark skin images...")
    dark_skin_df['image'] = process_images_in_batches(dark_skin_df['image_path'].tolist())

    # Step 3: Split dark skin data BEFORE augmentation to prevent data leakage
    dark_skin_train, dark_skin_test = train_test_split(
        dark_skin_df,
        test_size=0.4,
        stratify=dark_skin_df['label'],
        random_state=42
    )

    print(f"Dark skin split (before augmentation): {len(dark_skin_train)} training, {len(dark_skin_test)} testing")

    # Memory check point - print current memory usage
    try:
        print(f"Memory usage checkpoint 1: {process_memory_usage():.2f} GB")
    except:
        print("Memory monitoring not available")

    # Step 4: Augment dark skin training images
    print("\nAugmenting dark skin training images...")
    augmented_dark_skin_train = create_augmented_dark_skin_dataset(dark_skin_train)

    # Memory check point
    try:
        print(f"Memory usage checkpoint 2: {process_memory_usage():.2f} GB")
    except:
        print("Memory monitoring not available")

    # Step 5: Create TensorFlow datasets for ISIC data (efficient loading)
    print("\nCreating TensorFlow datasets for efficient loading...")

    # For ISIC train - prepare labels first
    isic_train_labels = tf.keras.utils.to_categorical(
        isic_train_df['class_id'].values,
        num_classes=len(CLASSES)
    )

    # Create train dataset that loads and processes images on-the-fly
    train_dataset = create_dataset(
        isic_train_df['image_path'].values,
        isic_train_labels,
        is_training=True
    )

    # For ISIC test data
    isic_test_labels = tf.keras.utils.to_categorical(
        isic_test_df['class_id'].values,
        num_classes=len(CLASSES)
    )

    test_dataset = create_dataset(
        isic_test_df['image_path'].values,
        isic_test_labels,
        is_training=False
    )

    # Step 6: Prepare dark skin test data
    # Ensure consistent float32 dtype
    X_dark_test = np.stack(dark_skin_test['image'].values).astype(np.float32) / 255.0
    y_dark_test = tf.keras.utils.to_categorical(dark_skin_test['class_id'], num_classes=len(CLASSES)).astype(np.float32)
    y_dark_test_labels = dark_skin_test['class_id'].values

    # Step 7: Prepare augmented dark skin training data
    # Ensure consistent float32 dtype
    X_dark_train_aug = np.stack(augmented_dark_skin_train['image'].values).astype(np.float32) / 255.0
    y_dark_train_aug = tf.keras.utils.to_categorical(
        augmented_dark_skin_train['class_id'],
        num_classes=len(CLASSES)
    ).astype(np.float32)

    # Create a tf.data.Dataset for dark skin data
    dark_skin_dataset = tf.data.Dataset.from_tensor_slices((X_dark_train_aug, y_dark_train_aug))
    dark_skin_dataset = dark_skin_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Clear RAM by removing dataframes we no longer need
    del dark_skin_df
    del dark_skin_train
    del augmented_dark_skin_train
    gc.collect()

    # Memory check point
    try:
        print(f"Memory usage checkpoint 3: {process_memory_usage():.2f} GB")
    except:
        print("Memory monitoring not available")

    # Print class distribution in dark skin test set
    print("\nDark skin test set class distribution:")
    for cls, idx in CLASS_MAPPING.items():
        count = np.sum(y_dark_test_labels == idx)
        print(f"  {cls}: {count} images")

    # Step 8: Create callbacks for training
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy',
            patience=2,
            factor=0.5,
            min_lr=0.00001,
            verbose=1
        ),
        # Add TensorBoard logging
        tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join(output_dir, 'logs'),
            histogram_freq=1
        )
    ]

    # Step 9: TRAIN BASELINE MODEL (ISIC data only)
    print("\nTraining baseline model on ISIC data only...")
    # Reset global policy before creating each model
    tf.keras.mixed_precision.set_global_policy('float32')
    baseline_model = create_model()

    # Print model summary
    baseline_model.summary()

    # Train baseline model
    baseline_history = baseline_model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=EPOCHS,
        callbacks=callbacks
    )

    # Plot training history
    plot_training_history(baseline_history, "Baseline Model", output_dir)

    # Step 10: TRAIN COMBINED MODEL (using separate approach instead of concatenate)
    print("\nTraining combined model on ISIC + augmented dark skin data...")
    # Reset global policy before creating each model
    tf.keras.mixed_precision.set_global_policy('float32')
    combined_model = create_model()

    # Instead of concatenating, use a custom training loop with both datasets
    # First train on ISIC data for a few epochs
    print("Phase 1: Training on ISIC data...")
    combined_model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=EPOCHS // 2,  # Train for half the epochs
        callbacks=callbacks
    )

    # Then fine-tune on dark skin data
    print("Phase 2: Fine-tuning on dark skin data...")
    combined_history = combined_model.fit(
        dark_skin_dataset,
        validation_data=test_dataset,
        epochs=EPOCHS // 2,  # Train for the remaining epochs
        callbacks=callbacks
    )

    # Plot training history
    plot_training_history(combined_history, "Combined Model", output_dir)

    # Step 11: EVALUATE MODELS ON DARK SKIN TEST DATA
    print("\nEvaluating both models on dark skin test data...")

    # Create a tf.data.Dataset for dark skin test for more efficient evaluation
    dark_test_dataset = tf.data.Dataset.from_tensor_slices((X_dark_test, y_dark_test))
    dark_test_dataset = dark_test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # Evaluate baseline model on dark skin test data
    baseline_dark_loss, baseline_dark_acc = baseline_model.evaluate(dark_test_dataset)
    baseline_preds = baseline_model.predict(dark_test_dataset)
    baseline_pred_classes = np.argmax(baseline_preds, axis=1)

    # Evaluate combined model on dark skin test data
    combined_dark_loss, combined_dark_acc = combined_model.evaluate(dark_test_dataset)
    combined_preds = combined_model.predict(dark_test_dataset)
    combined_pred_classes = np.argmax(combined_preds, axis=1)

    # Step 12: Generate and save confusion matrices
    baseline_cm = plot_confusion_matrix(
        y_dark_test_labels,
        baseline_pred_classes,
        CLASSES,
        title="Baseline Model Confusion Matrix",
        output_dir=output_dir
    )

    combined_cm = plot_confusion_matrix(
        y_dark_test_labels,
        combined_pred_classes,
        CLASSES,
        title="Combined Model Confusion Matrix",
        output_dir=output_dir
    )

    # Step 13: Generate and save classification reports
    baseline_report = save_classification_report(
        y_dark_test_labels,
        baseline_pred_classes,
        CLASSES,
        title="Baseline Model Classification Report",
        output_dir=output_dir
    )

    combined_report = save_classification_report(
        y_dark_test_labels,
        combined_pred_classes,
        CLASSES,
        title="Combined Model Classification Report",
        output_dir=output_dir
    )

    # Step 14: Store and analyze results
    # Print initial results
    print(f"Baseline model accuracy on dark skin test data: {baseline_dark_acc:.4f}")
    print(f"Combined model accuracy on dark skin test data: {combined_dark_acc:.4f}")

    # Store results
    baseline_results = {
        'overall': {'accuracy': baseline_dark_acc, 'loss': baseline_dark_loss},
        'per_class': {cls: {'correct': 0, 'total': 0} for cls in CLASSES}
    }

    combined_results = {
        'overall': {'accuracy': combined_dark_acc, 'loss': combined_dark_loss},
        'per_class': {cls: {'correct': 0, 'total': 0} for cls in CLASSES}
    }

    # Calculate per-class metrics
    for i, true_label in enumerate(y_dark_test_labels):
        true_class = CLASSES[true_label]
        baseline_correct = (baseline_pred_classes[i] == true_label)
        combined_correct = (combined_pred_classes[i] == true_label)

        # Update baseline results
        baseline_results['per_class'][true_class]['total'] += 1
        baseline_results['per_class'][true_class]['correct'] += int(baseline_correct)

        # Update combined results
        combined_results['per_class'][true_class]['total'] += 1
        combined_results['per_class'][true_class]['correct'] += int(combined_correct)

    # Calculate per-class accuracies
    for cls in CLASSES:
        for results in [baseline_results, combined_results]:
            if results['per_class'][cls]['total'] > 0:
                results['per_class'][cls]['accuracy'] = results['per_class'][cls]['correct'] / results['per_class'][cls]['total']
            else:
                results['per_class'][cls]['accuracy'] = 0

    # Plot model comparison
    plot_model_comparison(baseline_results, combined_results, CLASSES, output_dir)

    # Calculate improvement
    improvement = combined_results['overall']['accuracy'] - baseline_results['overall']['accuracy']
    print(f"\nOverall improvement from adding dark skin data: {improvement:.4f} " +
          f"({improvement*100:.1f}% {'increase' if improvement >= 0 else 'decrease'})")

    # Save the models
    baseline_model.save(os.path.join(output_dir, 'baseline_model.h5'))
    combined_model.save(os.path.join(output_dir, 'combined_model.h5'))
    print("Models saved to output directory")

    # Generate summary table of results
    summary_df = pd.DataFrame({
        'Metric': ['Overall Accuracy', 'Loss'] + [f"{cls} Accuracy" for cls in CLASSES],
        'Baseline Model': [baseline_results['overall']['accuracy'], baseline_results['overall']['loss']] +
                          [baseline_results['per_class'][cls]['accuracy'] for cls in CLASSES],
        'Combined Model': [combined_results['overall']['accuracy'], combined_results['overall']['loss']] +
                          [combined_results['per_class'][cls]['accuracy'] for cls in CLASSES],
        'Improvement': [improvement, baseline_results['overall']['loss'] - combined_results['overall']['loss']] +
                       [combined_results['per_class'][cls]['accuracy'] - baseline_results['per_class'][cls]['accuracy'] for cls in CLASSES]
    })

    # Save summary table
    summary_path = os.path.join(output_dir, 'model_comparison_summary.csv')
    summary_df.to_csv(summary_path, index=False)
    print(f"Summary table saved to {summary_path}")

    # Print summary table
    print("\nModel Comparison Summary:")
    print("-" * 80)
    print(summary_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
    print("-" * 80)

    # Calculate total runtime
    total_time = time.time() - start_time
    hours, remainder = divmod(total_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"\nTotal runtime: {int(hours)}h {int(minutes)}m {int(seconds)}s")

if __name__ == "__main__":
    try:
        # Add psutil for memory monitoring
        import psutil
    except ImportError:
        print("Installing psutil for memory monitoring...")
        import subprocess
        subprocess.check_call(["pip", "install", "psutil"])
        import psutil

    # Run the main function
    main()